In [ ]:
import math
import random
from typing import Any

import matplotlib.pyplot as plt
import pandas as pd

from pysorteddict import SortedDict

sorted_dict_lens = [100, 1_000, 10_000, 100_000, 1_000_000, 10_000_000]


def setup(sorted_dict_len: int, seed: float = math.pi) -> SortedDict:
    random.seed(seed)
    d = SortedDict()
    for _ in range(sorted_dict_len):
        d[random.random()] = random.random()
    return d


def _plot(df: pd.DataFrame, *, alpha: float, logy: bool, yunit: str) -> Any:
    fig, ax = plt.subplots()
    df.plot(
        alpha=alpha,
        ax=ax,
        logx=True,
        logy=logy,
        xlabel="Sorted Dictionary Length",
        xlim=(sorted_dict_lens[0] // 10, sorted_dict_lens[-1] * 10),
        ylabel="Average Execution Time",
    )
    ax.grid(which="major", linewidth=0.5, linestyle=":")
    ax.grid(which="minor", axis="y", linewidth=0.0625, linestyle="-")
    ax.minorticks_on()
    ax.legend(prop={"family": "JetBrains Mono"})
    # The warning about setting labels without setting ticks can be ignored. The plot isn't interactive.
    ax.set_yticklabels(f"{label.get_text()} {yunit}" for label in ax.get_yticklabels())
    plt.show()
    return fig


def plot(df: pd.DataFrame, *, alpha: float = 0.7, logy: bool = False, name: str = "", yunit: str = "s"):
    for style in ["light", "dark"]:
        with plt.style.context(f"{style}.mplstyle", after_reset=True):
            fig = _plot(df, alpha=alpha, logy=logy, yunit=yunit)
            fig.savefig(f"../../docs/_static/images/perf-{name}-{style}.svg")

# `contains`

In [ ]:
df = pd.DataFrame(index=sorted_dict_lens, columns=["0.00 in d", "0.33 in d", "0.67 in d", "1.00 in d"])
for i, sorted_dict_len in enumerate(sorted_dict_lens):
    d = setup(sorted_dict_len)
    for j, key in enumerate([0.00, 0.33, 0.67, 1.00]):
        %timeit -v bench key in d
        df.iat[i, j] = bench.average * 1e9

In [ ]:
display(df.T)
plot(df, name="contains", yunit="ns")

# `setitem`

In [ ]:
def set_del(d: SortedDict, keys: list[float]):
    for key in keys:
        d[key] = None
    for key in keys:
        del d[key]


keys_lens = [33, 67, 100]
df = pd.DataFrame(index=sorted_dict_lens, columns=[f"set_del(d, keys_{keys_len})" for keys_len in keys_lens])
for i, sorted_dict_len in enumerate(sorted_dict_lens):
    for j, keys_len in enumerate(keys_lens):
        d = setup(sorted_dict_len)
        keys = [random.random() for _ in range(keys_len)]
        %timeit -v bench set_del(d, keys)
        df.iat[i, j] = bench.average * 1e6

In [ ]:
display(df.T)
plot(df, name="setitem", yunit="μs")

# `iter`

In [ ]:
df = pd.DataFrame(index=sorted_dict_lens, columns=["for _ in d: pass", "for _ in reversed(d): pass"])
for i, sorted_dict_len in enumerate(sorted_dict_lens):
    d = setup(sorted_dict_len)
    %timeit -v bench for _ in d: pass
    df.iat[i, 0] = bench.average
    %timeit -v bench for _ in reversed(d): pass
    df.iat[i, 1] = bench.average

In [ ]:
display(df.T)
plot(df, logy=True, name="iter")